In [33]:
import pandas as pd
import os
import numpy as np
import scipy as ss
import scipy.stats
import datetime as dt
%matplotlib inline
import matplotlib.pyplot as plt
import cobra
from corda import reaction_confidence
from corda import CORDA
data = "/Users/annasintsova/git_repos/"\
        "HUTI-RNAseq/data/get_homologs_output/C50_S90_e0_/"\
        "run_C50_S90_e0__pan_C50_S90/"\
        "2018-02-26_pangenome_matrix_t0_crossRef.csv"

In [34]:
def assignConfidence(x):
    if x > 1.0:
        return 2 
    elif x < -1.5:
        return - 1
    else:
        return 0

In [35]:
def invnorm(x):
    return scipy.stats.norm.ppf((x.rank() -0.5)/x.count())

In [36]:
def getModel(strain):
    model_dir = "/Users/annasintsova/git_repos/"\
        "HUTI-RNAseq/analysis/fba/2017-12-16-model-2/data"
    model_path = os.path.join(model_dir, "{}_homologues/{}_model/{}_iML1515.xml".format(strain, strain, strain)) 
    return cobra.io.read_sbml_model(model_path)

In [38]:
def binNormRPKMs(data, strain):
    """
    Take in counts, select rpkms for correct strain, assign them confidence, 
    return confidence for ur and uti conditions (2 dictionaries)
    """
    zero_strains = ["HM1", "HM3", "HM6", "HM7"]
    new_strain = "HM0" + strain.split("HM")[-1] if strain in zero_strains else strain
        
    counts = pd.read_csv(data)
    rpkm = counts[["MG1655", "{}_UR_RPKM".format(new_strain), "{}_UTI_RPKM".format(new_strain)]]
    rpkm = rpkm.dropna(subset=["MG1655"])
    rpkm = rpkm[rpkm.MG1655 != "PARALOGS"]
    rpkm.set_index('MG1655', inplace=True)
    rpkm = rpkm.dropna()
    rpkm["{}_UR".format(strain)] = invnorm(rpkm["{}_UR_RPKM".format(new_strain)])
    rpkm["{}_UTI".format(strain)] = invnorm(rpkm["{}_UTI_RPKM".format(new_strain)])
    print(rpkm.head())
    confidences = {'UR_conf':list(map(assignConfidence, rpkm["{}_UR".format(strain)].values)),
              'UTI_conf':list(map(assignConfidence, rpkm["{}_UTI".format(strain)].values))}
    confidence_df = pd.DataFrame(confidences,index = rpkm.index)
    ur_conf = confidence_df.UR_conf.to_dict()
    uti_conf = confidence_df.UTI_conf.to_dict()
    return ur_conf, uti_conf
    #return rpkm

x,y = binNormRPKMs(data, "HM1")

a = list(x.values())
print(a.count(2))
print(a.count(-1))


        HM01_UR_RPKM  HM01_UTI_RPKM    HM1_UR   HM1_UTI
MG1655                                                 
b0002         109.11           7.42  0.324085 -0.025764
b0003         101.66           6.03  0.285648 -0.111864
b0004          55.64           4.29 -0.115007 -0.240792
b0006         142.44          31.56  0.496654  0.726625
b0007          18.12           1.40 -0.752278 -0.619864
509
214


In [ ]:



HM01_UTI_counts["invNORM_counts"] = invnorm(HM01_UTI_counts.HM01_UTI_RPKM)
HM01_UTI_counts.head()